# P4 — Operations / Support Analytics : 01_eda

In [ ]:
import pandas as pd, numpy as np
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

DB = 'sqlite:///../../da4.db'
TABLE = 'p4_tickets_stage'

engine = create_engine(DB)
with engine.begin() as conn:
    df = pd.read_sql(text(f'SELECT * FROM {TABLE}'), conn)

print('Shape:', df.shape)
display(df.head())
display(df.describe(numeric_only=True).T)
print('\nNull % (top):')
print((df.isna().mean()*100).sort_values(ascending=False).head(15))


# --- Distributions: first response & resolution hours
plt.figure(); df['first_response_hours'].dropna().plot(kind='hist', bins=30)
plt.title('First Response Hours'); plt.tight_layout(); plt.show()
plt.figure(); df['resolution_hours'].dropna().plot(kind='hist', bins=30)
plt.title('Resolution Hours'); plt.tight_layout(); plt.show()


# --- SLA by queue/priority
sla = (df.groupby(['queue','priority'], as_index=False)['within_sla'].mean()
         .rename(columns={'within_sla':'sla_met_pct'}))
sla['sla_met_pct'] *= 100
sla.sort_values(['queue','priority'], inplace=True)
sla


# --- Agent/queue table (top 10 slowest by resolution)
slow = (df.groupby(['agent_id','queue'], as_index=False)['resolution_hours']
          .mean().sort_values('resolution_hours', ascending=False).head(10))
slow
